In [0]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

In [0]:

from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, FloatType
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, CountVectorizer
from pyspark.ml import Pipeline
from pyspark.sql.functions import exp



from pyspark.sql.functions import col, exp, abs, when, lit
from pyspark.sql.types import *
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, CountVectorizer, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from math import exp
import matplotlib.pyplot as plt

In [0]:
spark = SparkSession.builder.appName('claim_count').getOrCreate()

In [0]:
df = spark.read.csv("/FileStore/tables/claims_data___Data__1_-1.csv",header=True, inferSchema= True) 

In [0]:
df.printSchema()

root
 |-- Reporting period end: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Data Item: string (nullable = true)
 |-- Class of business group: string (nullable = true)
 |-- Class of business category: string (nullable = true)
 |-- Class of business: string (nullable = true)
 |-- Reporting year basis: string (nullable = true)
 |-- Accident/underwriting year: integer (nullable = true)
 |-- Development year: integer (nullable = true)
 |-- Value: string (nullable = true)
 |-- Notes: string (nullable = true)



In [0]:
from pyspark.sql.functions import col

In [0]:
df.show()

+--------------------+-------+--------+--------------------+-----------------------+--------------------------+--------------------+--------------------+--------------------------+----------------+-----------+-----+
|Reporting period end|Subject|Category|           Data Item|Class of business group|Class of business category|   Class of business|Reporting year basis|Accident/underwriting year|Development year|      Value|Notes|
+--------------------+-------+--------+--------------------+-----------------------+--------------------------+--------------------+--------------------+--------------------------+----------------+-----------+-----+
|          12/31/2020|  Gross| Premium|Gross earned prem...|        Direct business|       Short-tail property|Commercial motor ...|       Accident year|                      2020|               0| 2864000000| null|
|          12/31/2020|  Gross| Premium|Gross earned prem...|        Direct business|       Short-tail property|Commercial motor ...|    

In [0]:
df.printSchema()

root
 |-- Reporting period end: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Data Item: string (nullable = true)
 |-- Class of business group: string (nullable = true)
 |-- Class of business category: string (nullable = true)
 |-- Class of business: string (nullable = true)
 |-- Reporting year basis: string (nullable = true)
 |-- Accident/underwriting year: integer (nullable = true)
 |-- Development year: integer (nullable = true)
 |-- Value: string (nullable = true)
 |-- Notes: string (nullable = true)



In [0]:
df1= df.filter((col("Data Item") == "Number of claims reported by class of business") & (col("Class of business") == "Domestic motor vehicle"))

In [0]:
df1.show()

+--------------------+----------------+----------------+--------------------+-----------------------+--------------------------+--------------------+--------------------+--------------------------+----------------+-------+--------------------+
|Reporting period end|         Subject|        Category|           Data Item|Class of business group|Class of business category|   Class of business|Reporting year basis|Accident/underwriting year|Development year|  Value|               Notes|
+--------------------+----------------+----------------+--------------------+-----------------------+--------------------------+--------------------+--------------------+--------------------------+----------------+-------+--------------------+
|          12/31/2020|Number of claims|Number of claims|Number of claims ...|        Direct business|       Short-tail property|Domestic motor ve...|       Accident year|                      2020|               0|1817000|                null|
|          12/31/2020|Nu

In [0]:
# shpae of data frame
print((df1.count(), len(df1.columns)))

(99, 12)


In [0]:
df2= df1.select(df1["Accident/underwriting year"],df1["Development year"], df1["Value"])

In [0]:
df2.show()

+--------------------------+----------------+-------+
|Accident/underwriting year|Development year|  Value|
+--------------------------+----------------+-------+
|                      2020|               0|1817000|
|                      2019|               1|2154000|
|                      2018|               2|2158000|
|                      2017|               3|2168000|
|                      2016|               4|2091000|
|                      2015|               5|2124000|
|                      2014|               6|1912000|
|                      2013|               7|1913000|
|                      2012|               8|1951000|
|                      2011|               9|   #N/A|
|                      2010|              10|1982000|
|                      2019|               0|2088000|
|                      2018|               1|2156000|
|                      2017|               2|2167000|
|                      2016|               3|2091000|
|                      2015|

In [0]:
new_df = df2.withColumn("Development year",
					df2["Accident/underwriting year"] + df2["Development year"])



In [0]:
new_df.show()

+--------------------------+----------------+-------+
|Accident/underwriting year|Development year|  Value|
+--------------------------+----------------+-------+
|                      2020|            2020|1817000|
|                      2019|            2020|2154000|
|                      2018|            2020|2158000|
|                      2017|            2020|2168000|
|                      2016|            2020|2091000|
|                      2015|            2020|2124000|
|                      2014|            2020|1912000|
|                      2013|            2020|1913000|
|                      2012|            2020|1951000|
|                      2011|            2020|   #N/A|
|                      2010|            2020|1982000|
|                      2019|            2019|2088000|
|                      2018|            2019|2156000|
|                      2017|            2019|2167000|
|                      2016|            2019|2091000|
|                      2015|

In [0]:
bootstrapped_df =  new_df.sample(withReplacement = True, fraction =100000.0)
bootstrapped_df.show()

+--------------------------+----------------+-------+
|Accident/underwriting year|Development year|  Value|
+--------------------------+----------------+-------+
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|

In [0]:
print((bootstrapped_df.count(), len(bootstrapped_df.columns)))

(9901029, 3)


In [0]:
bootstrapped_df.select(col("Value").cast("int").as("Value"))

  File "<command-3201196544375747>", line 1
    bootstrapped_df.select(col("Value").cast("int").as("Value"))
                                                    ^
SyntaxError: invalid syntax


In [0]:
bootstrapped_df.show()

+--------------------------+----------------+-------+
|Accident/underwriting year|Development year|  Value|
+--------------------------+----------------+-------+
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|

In [0]:
from pyspark.sql.types import IntegerType
bootstrapped_df = bootstrapped_df.withColumn("Value", bootstrapped_df["Value"].cast(IntegerType()))

In [0]:
bootstrapped_df.show()

+--------------------------+----------------+-------+
|Accident/underwriting year|Development year|  Value|
+--------------------------+----------------+-------+
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|            2020|1817000|
|                      2020|

In [0]:

final_df= bootstrapped_df.groupBy("Accident/underwriting year","Development year") \
    .sum("Value")
    

In [0]:
final_df.show(100)

+--------------------------+----------------+------------+
|Accident/underwriting year|Development year|  sum(Value)|
+--------------------------+----------------+------------+
|                      2014|            2016|176082722000|
|                      2011|            2014|172565504000|
|                      2008|            2013|149773052000|
|                      2010|            2015|176085728000|
|                      2006|            2013|131920686000|
|                      2003|            2012|113926922000|
|                      2011|            2012|174615347000|
|                      2005|            2015|128905830000|
|                      2012|            2016|178810728000|
|                      2013|            2014|175476861000|
|                      2004|            2014|122404992000|
|                      2015|            2015|191238684000|
|                      2005|            2012|129103380000|
|                      2004|            2013|12194616000

In [0]:
final_df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").option("delimiter", "\t").option("compression", "gzip").save("dbfs:/FileStore/df/final_df.csv")

In [0]:
input_path = "dbfs:/FileStore/df/final_df.csv"

In [0]:
in_data = spark.read.csv(input_path, header=True)

In [0]:
display(in_data)

Accident/underwriting year Development year sum(Value)
2014 2016 176082722000
2011 2014 172565504000
2008 2013 149773052000
2010 2015 176085728000
2006 2013 131920686000
2003 2012 113926922000
2011 2012 174615347000
2005 2015 128905830000
2012 2016 178810728000
2013 2014 175476861000
